<a href="https://colab.research.google.com/github/monjjjjj/orchid_classification/blob/main/orchid_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python import keras
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 
from tensorflow import keras
from google.colab import drive
from google.colab.patches import cv2_imshow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras import initializers, optimizers
from keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50, ResNet152
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [ ]:
# google drive mounted
drive.mount('/content/drive')

In [ ]:
# data path
train_dir = "/content/drive/MyDrive/orchid_classification/training/"
label_dir = "/content/drive/MyDrive/orchid_classification/label/label.csv"

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

In [ ]:
# read data
def load_data():
  x_train = []
  y_train = []
  df = pd.read_csv(label_dir, index_col = "filename")
  for p in os.listdir(train_dir):
    if p != "label.csv":
      path = train_dir + p
      image = cv2.imread(path)
      image = image.astype("float") / 255.0
      image = cv2.resize(image, IMG_SIZE, interpolation = cv2.INTER_AREA)
      x_train.append(np.array(image))
      y_train.append(df.loc[str(p),"category"])
  return x_train,y_train



In [ ]:
# load data
x_train, y_train = load_data()

# one-hot encoding
x_train = np.array(x_train)
num_classes = 219
y_train = np_utils.to_categorical(y_train, num_classes, dtype = 'float32')

In [ ]:
# debug
#print(x_train[1:5])
#print(y_train[1:5])
#print(x_train.shape)
#print(y_train.shape)

In [ ]:
# spilt data
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train, train_size=0.5, test_size=0.5, random_state=0, stratify = y_train)

In [ ]:
'''
# data augmentation
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

data_augmentation(x_train)
'''

In [ ]:
#data augmentation
datagen = ImageDataGenerator(
        featurewise_center = False,  # set input mean to 0 over the dataset
        samplewise_center = False,  # set each sample mean to 0
        featurewise_std_normalization = False,  # divide inputs by std of the dataset
        samplewise_std_normalization = False,  # divide each input by its std
        zca_whitening = False,  # apply ZCA whitening
        rotation_range = 0.5,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip = True)  # randomly flip images
datagen.fit(x_train1)
#datagen.fit(x_test1)

In [ ]:
# build model for ResNet50
def build_model():
  model = Sequential()
  model.add(ResNet50(include_top=False, weights='imagenet', input_tensor=None, pooling='avg', classes=219, input_shape=(224, 224, 3)))
  #model.add(Flatten())
  model.add(Dense(219,activation='softmax'))

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  print(model.summary())
  keras.utils.plot_model(model, show_shapes=True, dpi=64, to_file='model2.png')

  return model

In [ ]:
# training for ResNet50
model = build_model()

#train_history = model.fit(x=x_train, y=y_train, batch_size= BATCH_SIZE, epochs=20, verbose=1)
train_history = model.fit_generator(datagen.flow(x_train1, y_train1, batch_size = BATCH_SIZE), validation_data=(x_test1, y_test1), epochs = 50, verbose = 1)


In [ ]:
# build model for ResNet152
def build_model1():
  model = Sequential()
  model.add(ResNet152(include_top=False, weights='imagenet', input_tensor=None, pooling='avg', classes=219, input_shape=(224, 224, 3)))
  model.add(Flatten())
  model.add(Dense(219,activation='softmax'))

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  print(model.summary())
  keras.utils.plot_model(model, show_shapes=True, dpi=64, to_file='model2.png')

  return model

In [ ]:
# training for ResNet152
model1 = build_model1()

train_history1 = model1.fit(x=x_train, y=y_train, batch_size= BATCH_SIZE, epochs=20, verbose=1)


In [ ]:
# build model for VGG16
def build_model2():
  model = Sequential()
  model.add(VGG16(include_top= False, weights='imagenet', input_tensor=None, pooling='avg', classes=219, input_shape=(224, 224, 3)))
  model.add(Flatten())
  model.add(Dense(219,activation='softmax'))

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  print(model.summary())
  keras.utils.plot_model(model, show_shapes=True, dpi=64, to_file='model2.png')

  return model

In [ ]:
#training for VGG16
model2 = build_model2()

train_history2 = model2.fit(x=x_train, y=y_train, batch_size= BATCH_SIZE, epochs=30, verbose=1)

In [ ]:
print(train_history.history.keys())
#print(train_history1.history.keys())

In [ ]:
def show_train_history(train_history,train,validation):
  
  if train == 'accuracy':
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
  else:
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')

  plt.legend(['train','validation'],loc='upper left')
  plt.show()

show_train_history(train_history,'accuracy','val_accuracy')
show_train_history(train_history,'loss','val_loss')
#show_train_history(train_history1,'accuracy','val_accuracy')
#show_train_history(train_history1,'loss','val_loss')
#show_train_history(train_history2,'accuracy','val_accuracy')
#show_train_history(train_history2,'loss','val_loss')


In [ ]:
# summarize history for accuracy
#x = np.arange(20)
plt.plot(train_history.history['accuracy'], label=ResNet50)
#plt.plot(x, train_history1.history['accuracy'], label=ResNet152)
#plt.plot(train_history2.history['accuracy'], label=VGG16)

plt.plot(train_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Training', 'Validation'], loc='lower right') 
plt.show()

In [ ]:
# summarize history for loss
# x=np.arange(20)
plt.plot(train_history.history['loss'])
#plt.plot(train_history1.history['loss'])
#plt.plot(train_history2.history['loss'])

plt.plot(train_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Training', 'Validation'], loc='upper right') 
plt.show()

In [ ]:
#testing
scores=model.evaluate(x_test1,y_test1)
print()
print('accuracy:',scores[1])